In [1]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2  # Using MobileNetV2 as a modern substitute
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Download the dataset
path = kagglehub.dataset_download("jagdishchakole1/mango-data")
print("Path to dataset files:", path)

# Define data directories
train_dir = os.path.join(path, "Mango_leaf_disease1/train")
val_dir = os.path.join(path, "Mango_leaf_disease1/val")
test_dir = os.path.join(path, "Mango_leaf_disease1/test")



100%|██████████| 104M/104M [00:00<00:00, 177MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jagdishchakole1/mango-data/versions/1


In [2]:
# Create data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Just rescaling for validation and test sets
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the generators
input_shape = (224, 224)  # Standard input size for MobileNetV2

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode='categorical'
)

Found 2800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [3]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 400 images belonging to 8 classes.


In [4]:
# Get the number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")
print("Class mapping:", train_generator.class_indices)

# Create the base model from the pre-trained MobileNetV2
base_model = MobileNetV2(
    weights='imagenet',  # Using pre-trained weights from ImageNet
    include_top=False,
    input_shape=(224, 224, 3)
)

Number of classes: 8
Class mapping: {'Anthracnose': 0, 'Bacterial Canker': 1, 'Cutting Weevil': 2, 'Die Back': 3, 'Gall Midge': 4, 'Healthy': 5, 'Powdery Mildew': 6, 'Sooty Mould': 7}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [6]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

# Add callbacks
checkpoint = ModelCheckpoint(
    'mobilenet_best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 3,577,928 (13.65 MB)

 Trainable params: 1,319,944 (5.04 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[checkpoint, early_stopping]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# Save the final model
model.save('mango_disease_mobilenet_model.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.4906 - loss: 1.4707

87/87 ━━━━━━━━━━━━━━━━━━━━ 51s 459ms/step - accuracy: 0.4927 - loss: 1.4649 - val_accuracy: 0.9538 - val_loss: 0.2660
Epoch 2/20
 1/87 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9062 - loss: 0.3810

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9062 - loss: 0.3810 - val_accuracy: 0.9538 - val_loss: 0.2642
Epoch 3/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.8798 - loss: 0.3815

...

87/87 ━━━━━━━━━━━━━━━━━━━━ 39s 364ms/step - accuracy: 0.9418 - loss: 0.2013 - val_accuracy: 0.9775 - val_loss: 0.0969
Epoch 6/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.1145 - val_accuracy: 0.9775 - val_loss: 0.0961
Epoch 7/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.9532 - loss: 0.1678

87/87 ━━━━━━━━━━━━━━━━━━━━ 41s 387ms/step - accuracy: 0.9532 - loss: 0.1677 - val_accuracy: 0.9837 - val_loss: 0.0747
Epoch 8/20
 1/87 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9375 - loss: 0.1975

87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9375 - loss: 0.1975 - val_accuracy: 0.9862 - val_loss: 0.0737
Epoch 9/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 38s 367ms/step - accuracy: 0.9675 - loss: 0.1211 - val_accuracy: 0.9787 - val_loss: 0.0704
Epoch 10/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9688 - loss: 0.1039 - val_accuracy: 0.9775 - val_loss: 0.0712
Epoch 11/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.9698 - loss: 0.1046

87/87 ━━━━━━━━━━━━━━━━━━━━ 32s 374ms/step - accuracy: 0.9698 - loss: 0.1046 - val_accuracy: 0.9912 - val_loss: 0.0485
Epoch 12/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9688 - loss: 0.1587 - val_accuracy: 0.9912 - val_loss: 0.0486
Epoch 13/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 38s 359ms/step - accuracy: 0.9788 - loss: 0.0773 - val_accuracy: 0.9825 - val_loss: 0.0499
Epoch 14/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 1.0000 - loss: 0.0671 - val_accuracy: 0.9837 - val_loss: 0.0472
Epoch 15/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 40s 368ms/step - accuracy: 0.9778 - loss: 0.0798 - val_accuracy: 0.9837 - val_loss: 0.0449
Epoch 16/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 1.0000 - loss: 0.0472 - val_accuracy: 0.9837 - val_loss: 0.0449
Epoch 17/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 39s 356ms/step - accuracy: 0.9798 - loss: 0.0736 - val_accuracy: 0.9887 - val_loss: 0.0446
Epoch 18/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 1.0000 - loss: 0.0692 - val_accuracy: 0.9862

Test accuracy: 0.9925
